# 什么是内容向量召回？
内容向量召回是基于物品的内容特征（如文本、图像、属性等）构建向量表示，通过向量相似度计算来实现物品召回的方法。

- 核心思想
    - 将物品内容转化为数值化向量
    - 在向量空间中度量物品相似度
    - 基于用户历史偏好寻找相似内容

---
## 1. 概念与作用（为什么要用内容向量）

- 内容向量：把物品（或用户）的内容/属性（文本、分类特征、图像、数值属性等）映射为一个低维实数向量（embedding），用向量表示语义/属性相似性。

- 在召回层的作用：通过向量相似度（如余弦/内积）快速召回与用户已交互物品或用户画像相似的一堆候选（Candidate），通常作为召回（Recall）阶段的一种方式（与`协同过滤召回`、`行为共现召回`、`热门`/`规则召回`并列）。

- 优点：解决新物品冷启动、易解释（基于内容相似）、能结合文本/图像/结构化属性做混合召回；工程上可离线计算并用向量索引（ANN）实时检索。

- 缺点：内容特征质量决定效果；单纯基于内容可能缺乏协同信号（个性化弱），通常与协同信号融合使用。

---
## 2. 内容向量的来源（常见类型）

- 稀疏` TF-IDF `/` BoW `向量（文本标题/描述）

- 预训练词向量或句向量（word2vec 平均、Doc2Vec、sentence-transformers / BERT）

- 图像特征向量（预训练 CNN 的池化输出）

- 结构化 / 类别特征` embedding`（类别 one-hot 后 embedding 表）

- 混合多模态向量（文本 + 图像 + 结构化拼接或融合后降维）

- 学习得来的向量（Supervised / Metric Learning）：用有监督任务或对比学习训练得到的 embedding（更适合下游召回/排序）

---
## 3. 向量构建方法（从简单到复杂）

- `TF-IDF`（简单、快速）：对 `title`/`description` 建模，适合快速 `baseline`，配合 `MinHash` / `LSH` 做近似检索。

- 平均词向量（word2vec avg）：用预训练词向量平均或加权平均（按 TF-IDF 权重）得到句子/描述向量。

- Sentence-BERT / Transformer Embeddings（效果好）：直接把句子/长文本映射成 384/768 维向量，语义捕获更强。

- 图像 Embedding：用 ResNet、EfficientNet 等预训练网络，取 GAP 后向量。

- 融合策略：简单拼接（concatenate）→ 可做 FC 层压缩；或用注意力/自适应融合（learned fusion）。

- 训练策略：

    - 监督（Classification/CTR）训练：把 item embedding 作为特征，训练目标是点击/转化 —— embedding 会更有任务相关性。
    
    - 对比学习（Siamese/Triplet/InfoNCE）：把“相似交互对”拉近，不相似对拉远（适用于无标签或弱标签场景）。

---
## 4. 用户画像（User Profile）如何生成

- 简单加权平均：用户向量 = 已交互 `item` 向量的加权平均（权重按 recency、行为类型、评分、购买金额等）。

- 向量池化：`max pooling` / `attention pooling`（learned attention 权重），可更好地聚焦用户的偏好。

- 序列模型：用 `RNN`/ `Transformer` 对用户最近行为序列编码，输出用户向量（session-based）。

---
## 5. 召回流程（工程级）

- 离线阶段：抽取 `item` 元数据 → 生成 `item` 向量（文本/图像/融合）→ 可选降维（`PCA`）→ 构建` ANN `索引（FAISS/HNSW/Milvus）→ 保存索引 + 向量表。

- 在线阶段：实时计算/读取用户向量（或根据用户最近行为做加权平均）→ 在` ANN `索引中检索` Top-K`（K=100~200 常见）→ 过滤（已交互）→ 将候选送入排序模型。

- 增量更新：新` item `生成向量后可增量插入索引或批量重建索引（根据索引类型支持）。

---
## 6. 相似度与 ANN 索引（细节）

- 相似度度量：余弦相似度（常用），内积（dot）用于未归一化或训练时直接优化内积。

- 若用内积但想等价于余弦：对向量做` L2 `归一化。

- `ANN`（近似最近邻）建议：

    - 中小规模（百万级）可用 `HNSW`（高召回、低延迟）。
    
    - 大规模（千万/亿）可用` IVF+PQ`（倒排+乘积量化）节省内存。
    
    - 推荐库：FAISS（GPU/CPU）、Annoy、HNSWlib、Milvus、Weaviate（向量 DB）。

- 索引参数常见建议（经验值，需调优）：

    - embedding_dim: 128/256/384/768（取决于模型）。
    
    - topK_recall: 100 ~ 200（一般召回层输出 100~200 个候选交给排序）。
    
    - HNSW：M=16~64，efSearch=100~500（越大召回越高、延迟越高）。
    
    - IVF+PQ：nlist=1k~10k, m(subquantizers)=8~16, nbits=8。

---
## 7. 向量压缩与存储

- 量化（PQ）：降低内存，用在大规模库。

- 降维（PCA / TruncatedSVD）：把 768D 压到 128D，保留大部分信息同时加速检索。

- 数据类型：存 float32 常见，float16 可节省内存但注意数值精度。

---
## 8. 冷启动 & 稀疏性问题

新` item`：只要有 metadata（title、category、image），就能直接生成向量并召回——内容向量是解决新物品冷启动的利器。

- 新用户：用热门/策略推荐或用 onboarding 问卷获取初始偏好，也可以基于 session 的短期行为生成临时用户向量。

- 长尾 `item`：可能少交互，内容相似性仍能帮助召回长尾，但效果依赖内容质量。

---
## 9. 评估（召回层相关指标）

- `Recall@K`：召回阶段最关键（衡量召回能否覆盖用户将要点击/购买的物品）。

- `Coverage`、`Novelty`、`Diversity`：关注候选集的广度与新颖性。

- `Latency` / `QPS`：线上召回必须满足延迟要求（通常 10s-100s ms）。

---
## 10. 实践最佳实践与工程要点（Checklist）

- 对向量做 L2 归一化（如果以余弦为准）；统一训练与检索时的度量。

- `Top-K `稳定化：召回 K 不宜太小，推荐层和排序层共同决定最终结果。

- 索引热更新：选择支持增量插入或实现短周期批量重建。

- 时间衰减：用户画像加时间权重以增强时效性。

- 多模态融合：如果有图文，先在各自空间做 embedding(特征提取)，再用 FC(全连接层)/Attention(注意力机制) 融合并归一化。

- 对比学习 / 熟化训练：如果有交互数据，优先用对比损失训练向量以提高检索质量（InfoNCE、Triplet）。

- A/B 测试：线上验证不同向量/索引/检索策略对` CTR`、`转化`的真实影响。

- 监控：召回覆盖率、查询延迟、索引错误率、召回质量变化（随时间）。

---
## 内容向量的构建方法
文本内容向量化  
- TF-IDF向量

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class TFIDFContentVector:
    def __init__(self, max_features=5000):
        self.vectorizer = TfidfVectorizer(
            max_features=max_features,
            stop_words='english',
            ngram_range=(1, 2)
        )
        self.item_vectors = None
    
    def fit(self, items_data):
        """训练TF-IDF向量化器"""
        # items_data: {item_id: "物品描述文本"}
        texts = list(items_data.values())
        self.item_vectors = self.vectorizer.fit_transform(texts)
        self.item_ids = list(items_data.keys())
        self.id_to_idx = {id_: idx for idx, id_ in enumerate(self.item_ids)}
    
    def get_similar_items(self, item_id, top_k=10):
        """获取相似物品"""
        if item_id not in self.id_to_idx:
            return []
        
        idx = self.id_to_idx[item_id]
        item_vector = self.item_vectors[idx]
        
        # 计算余弦相似度
        similarities = cosine_similarity(item_vector, self.item_vectors).flatten()
        
        # 获取最相似的物品
        similar_indices = similarities.argsort()[::-1][1:top_k+1]  # 排除自身
        
        results = []
        for sim_idx in similar_indices:
            results.append({
                'item_id': self.item_ids[sim_idx],
                'similarity': similarities[sim_idx]
            })
        
        return results

Word2Vec/Doc2Vec向量

In [ ]:
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np

class Doc2VecContentVector:
    def __init__(self, vector_size=100, window=5, min_count=2):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.model = None
        self.item_vectors = {}
    
    def preprocess_text(self, text):
        """文本预处理"""
        # 简单的分词处理
        return text.lower().split()
    
    def fit(self, items_data):
        """训练Doc2Vec模型"""
        # 准备训练数据
        tagged_docs = []
        for item_id, text in items_data.items():
            words = self.preprocess_text(text)
            tagged_docs.append(TaggedDocument(words, [item_id]))
        
        # 训练模型
        self.model = Doc2Vec(
            vector_size=self.vector_size,
            window=self.window,
            min_count=self.min_count,
            workers=4,
            epochs=20
        )
        
        self.model.build_vocab(tagged_docs)
        self.model.train(tagged_docs, 
                        total_examples=self.model.corpus_count,
                        epochs=self.model.epochs)
        
        # 存储物品向量
        for item_id in items_data.keys():
            self.item_vectors[item_id] = self.model.dv[item_id]
    
    def get_similar_items(self, item_id, top_k=10):
        """获取相似物品"""
        if item_id not in self.item_vectors:
            return []
        
        similar_items = self.model.dv.most_similar(item_id, topn=top_k)
        
        return [{'item_id': item, 'similarity': sim} for item, sim in similar_items]

多模态内容向量  --- 结合文本和属性特征

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA

class MultiModalContentVector:
    def __init__(self, text_dim=100, numeric_dim=10, categorical_dim=20):
        self.text_dim = text_dim
        self.numeric_dim = numeric_dim
        self.categorical_dim = categorical_dim
        
    def fit(self, items_df):
        """处理多模态特征"""
        self.items_df = items_df
        self.item_vectors = {}
        
        # 1. 处理文本特征
        text_vectors = self._process_text_features(items_df['description'])
        
        # 2. 处理数值特征
        numeric_vectors = self._process_numeric_features(items_df[['price', 'rating', 'sales']])
        
        # 3. 处理分类特征
        categorical_vectors = self._process_categorical_features(items_df[['category', 'brand']])
        
        # 4. 特征融合
        for idx, item_id in enumerate(items_df['item_id']):
            combined_vector = np.concatenate([
                text_vectors[idx],
                numeric_vectors[idx],
                categorical_vectors[idx]
            ])
            self.item_vectors[item_id] = combined_vector
    
    def _process_text_features(self, texts):
        """处理文本特征"""
        from sklearn.feature_extraction.text import TfidfVectorizer
        
        vectorizer = TfidfVectorizer(max_features=self.text_dim)
        text_vectors = vectorizer.fit_transform(texts).toarray()
        
        # 降维（如果需要）
        if text_vectors.shape[1] > self.text_dim:
            pca = PCA(n_components=self.text_dim)
            text_vectors = pca.fit_transform(text_vectors)
        
        return text_vectors
    
    def _process_numeric_features(self, numeric_df):
        """处理数值特征"""
        scaler = StandardScaler()
        scaled_features = scaler.fit_transform(numeric_df)
        return scaled_features
    
    def _process_categorical_features(self, categorical_df):
        """处理分类特征"""
        encoded_features = []
        
        for column in categorical_df.columns:
            le = LabelEncoder()
            encoded_col = le.fit_transform(categorical_df[column])
            # One-hot编码
            onehot = pd.get_dummies(encoded_col, prefix=column)
            encoded_features.append(onehot.values)
        
        # 合并所有分类特征
        categorical_vectors = np.concatenate(encoded_features, axis=1)
        
        # 降维
        if categorical_vectors.shape[1] > self.categorical_dim:
            pca = PCA(n_components=self.categorical_dim)
            categorical_vectors = pca.fit_transform(categorical_vectors)
        
        return categorical_vectors

## 基于内容向量的召回系统
 用户画像构建

In [ ]:
class UserProfileVector:
    def __init__(self, content_vector_model):
        self.content_vector_model = content_vector_model
        self.user_profiles = {}
    
    def update_user_profile(self, user_id, interacted_items, weights=None):
        """更新用户画像"""
        if weights is None:
            weights = np.ones(len(interacted_items))
        
        user_vector = None
        valid_items = 0
        
        for item_id, weight in zip(interacted_items, weights):
            if item_id in self.content_vector_model.item_vectors:
                item_vector = self.content_vector_model.item_vectors[item_id]
                
                if user_vector is None:
                    user_vector = np.zeros_like(item_vector)
                
                user_vector += item_vector * weight
                valid_items += 1
        
        if valid_items > 0:
            user_vector /= valid_items  # 平均 pooling
            self.user_profiles[user_id] = user_vector
    
    def get_user_recommendations(self, user_id, all_item_ids, top_k=20):
        """基于用户画像获取推荐"""
        if user_id not in self.user_profiles:
            return []
        
        user_vector = self.user_profiles[user_id]
        similarities = []
        
        for item_id in all_item_ids:
            if item_id in self.content_vector_model.item_vectors:
                item_vector = self.content_vector_model.item_vectors[item_id]
                similarity = cosine_similarity([user_vector], [item_vector])[0][0]
                similarities.append((item_id, similarity))
        
        # 排序并返回Top-K
        similarities.sort(key=lambda x: x[1], reverse=True)
        return similarities[:top_k]

实时向量召回

In [ ]:
import faiss  # Facebook开源的向量相似度搜索库

class FaissVectorRecall:
    def __init__(self, dimension):
        self.dimension = dimension
        self.index = faiss.IndexFlatIP(dimension)  # 内积相似度
        self.item_id_to_index = {}
        self.index_to_item_id = {}
    
    def build_index(self, item_vectors):
        """构建向量索引"""
        vectors = []
        
        for idx, (item_id, vector) in enumerate(item_vectors.items()):
            vectors.append(vector)
            self.item_id_to_index[item_id] = idx
            self.index_to_item_id[idx] = item_id
        
        # 归一化向量（用于余弦相似度）
        vectors = np.array(vectors).astype('float32')
        faiss.normalize_L2(vectors)
        
        self.index.add(vectors)
        print(f"索引构建完成，共{self.index.ntotal}个向量")
    
    def search_similar_items(self, query_vector, top_k=10):
        """搜索相似物品"""
        if isinstance(query_vector, list):
            query_vector = np.array(query_vector).astype('float32')
        
        # 归一化查询向量
        faiss.normalize_L2(query_vector.reshape(1, -1))
        
        # 搜索
        similarities, indices = self.index.search(query_vector.reshape(1, -1), top_k)
        
        results = []
        for i, (similarity, index) in enumerate(zip(similarities[0], indices[0])):
            if index != -1:  # 有效结果
                item_id = self.index_to_item_id[index]
                results.append({
                    'item_id': item_id,
                    'similarity': float(similarity),
                    'rank': i + 1
                })
        
        return results
    
    def search_by_item(self, item_id, top_k=10):
        """基于物品搜索相似物品"""
        if item_id not in self.item_id_to_index:
            return []
        
        index = self.item_id_to_index[item_id]
        # 获取物品向量
        item_vector = self.index.reconstruct(index)
        
        return self.search_similar_items(item_vector, top_k + 1)[1:]  # 排除自身

## 完整的内容向量召回系统

In [ ]:
class ContentBasedRecallSystem:
    def __init__(self, vector_dim=300):
        self.vector_dim = vector_dim
        self.vector_model = None
        self.faiss_index = None
        self.user_profiles = {}
    
    def initialize_system(self, items_data):
        """初始化系统"""
        print("开始构建内容向量模型...")
        
        # 1. 构建内容向量
        self.vector_model = MultiModalContentVector()
        self.vector_model.fit(items_data)
        
        # 2. 构建向量索引
        self.faiss_index = FaissVectorRecall(self.vector_dim)
        self.faiss_index.build_index(self.vector_model.item_vectors)
        
        print("内容向量召回系统初始化完成")
    
    def update_user_behavior(self, user_id, interacted_items, behavior_type='click'):
        """更新用户行为"""
        if user_id not in self.user_profiles:
            self.user_profiles[user_id] = {
                'clicked_items': [],
                'purchased_items': [],
                'last_update': None
            }
        
        # 根据行为类型分配权重
        weights = {
            'click': 1.0,
            'purchase': 3.0,
            'favorite': 2.0
        }
        
        weight = weights.get(behavior_type, 1.0)
        
        # 更新用户画像
        user_profile = UserProfileVector(self.vector_model)
        
        if behavior_type == 'click':
            self.user_profiles[user_id]['clicked_items'].extend(
                [(item, weight) for item in interacted_items]
            )
        elif behavior_type == 'purchase':
            self.user_profiles[user_id]['purchased_items'].extend(
                [(item, weight) for item in interacted_items]
            )
        
        # 重新计算用户向量
        all_interactions = (self.user_profiles[user_id]['clicked_items'] + 
                          self.user_profiles[user_id]['purchased_items'])
        
        if all_interactions:
            items, weights = zip(*all_interactions)
            user_profile.update_user_profile(user_id, items, weights)
    
    def get_recommendations(self, user_id, recall_num=50):
        """获取推荐结果"""
        if user_id not in self.user_profiles or not self.user_profiles[user_id]['clicked_items']:
            # 冷启动用户：返回热门物品或随机物品
            return self._get_cold_start_recommendations(recall_num)
        
        # 基于用户画像的召回
        user_profile = UserProfileVector(self.vector_model)
        items, weights = zip(*self.user_profiles[user_id]['clicked_items'])
        user_profile.update_user_profile(user_id, items, weights)
        
        user_vector = user_profile.user_profiles[user_id]
        recommendations = self.faiss_index.search_similar_items(user_vector, recall_num)
        
        return recommendations
    
    def _get_cold_start_recommendations(self, recall_num):
        """冷启动推荐策略"""
        # 可以基于物品流行度、新鲜度等进行召回
        all_item_ids = list(self.vector_model.item_vectors.keys())
        np.random.shuffle(all_item_ids)
        return [{'item_id': item_id, 'similarity': 0.0} for item_id in all_item_ids[:recall_num]]
    
    def get_similar_items(self, item_id, top_k=10):
        """获取相似物品（用于相关推荐）"""
        return self.faiss_index.search_by_item(item_id, top_k)

## 实际应用示例
电影推荐系统

In [ ]:
# 示例数据准备
movies_data = {
    'item_id': [1, 2, 3, 4, 5],
    'title': ['The Godfather', 'The Dark Knight', 'Pulp Fiction', 'Forrest Gump', 'Inception'],
    'description': [
        'Crime family drama about the Corleone family',
        'Batman battles the Joker in Gotham City',
        'Interconnected stories of criminals in Los Angeles',
        'Life story of a man with low IQ but good heart',
        'Dream thieves perform corporate espionage through dreams'
    ],
    'genre': ['Crime', 'Action', 'Crime', 'Drama', 'Sci-Fi'],
    'rating': [9.2, 9.0, 8.9, 8.8, 8.7]
}

# 创建推荐系统
recall_system = ContentBasedRecallSystem()
recall_system.initialize_system(pd.DataFrame(movies_data))

# 模拟用户行为
recall_system.update_user_behavior('user1', [1, 2], 'click')  # 用户点击了教父和黑暗骑士

# 获取推荐
recommendations = recall_system.get_recommendations('user1', 10)
print("为用户推荐的电影:")
for rec in recommendations:
    print(f"电影ID: {rec['item_id']}, 相似度: {rec['similarity']:.3f}")

# 获取相似电影
similar_movies = recall_system.get_similar_items(1, 5)  # 与教父相似的电影
print("\n与《教父》相似的电影:")
for movie in similar_movies:
    print(f"电影ID: {movie['item_id']}, 相似度: {movie['similarity']:.3f}")